# Seleção de Hinos Similares com TOPSIS
Este notebook utiliza o método TOPSIS para selecionar hinos similares a partir de um hino de referência, considerando múltiplos critérios extraídos das análises anteriores.

O objetivo é combinar informações de similaridade, categorias, clusters e tópicos para sugerir hinos próximos ao perfil desejado.

---
**Conteúdo do notebook:**
- Carregamento dos dados e matrizes de similaridade
- Construção da matriz de decisão para o TOPSIS
- Definição dos pesos e critérios
- Aplicação do método TOPSIS
- Exibição dos hinos mais similares sugeridos

Este material é público e pode ser compartilhado para fins de pesquisa, estudo ou divulgação cultural.

# Parte 6 - TOPSIS pra seleção de hinos similares

Seleção de hinos similares a partir de um hino de referência, utilizando o método TOPSIS, considerando as matrizes de similaridade, categorias, clusterings, e tópicos.

---

**A seguir:** Carregamento dos dados dos hinos e das matrizes de similaridade calculadas nas etapas anteriores.

In [1]:
import pandas as pd

hinos_analise: pd.DataFrame = pd.read_pickle(
    "../assets/hinos_analise_embeddings_complete.pkl"
)
similarity_matrix_words: pd.DataFrame = pd.read_pickle(
    "../assets/similarity_matrix_word_embeddings_tfidf.pkl"
)
similarity_matrix_sent: pd.DataFrame = pd.read_pickle(
    "../assets/similarity_matrix_sentence_embeddings.pkl"
)

---

**A seguir:** Construção da matriz de decisão, onde cada alternativa representa um hino e os critérios são as similaridades, categorias, clusters e tópicos em relação ao hino de referência.

A matriz de decisão é construída com relação a um hino de referência, e as alternativas são os demais hinos.

---

**A seguir:** Cálculo das similaridades entre o hino de referência e os demais hinos, adicionando essas informações à matriz de decisão.

In [2]:
hino_sample = hinos_analise.sample().iloc[0]
hinos_restantes = hinos_analise[hinos_analise.index != hino_sample.name].copy()
# se categoria_id é a mesma de sample, então 1, senão 0
hinos_restantes["categoria_id"] = (
    hinos_restantes["categoria_id"] == hino_sample["categoria_id"]
).astype(int)
hinos_restantes["word_cluster"] = (
    hinos_restantes["word_cluster"] == hino_sample["word_cluster"]
).astype(int)
hinos_restantes["NMF_topic"] = (
    hinos_restantes["NMF_topic"] == hino_sample["NMF_topic"]
).astype(int)
hinos_restantes["sent_cluster"] = (
    hinos_restantes["sent_cluster"] == hino_sample["sent_cluster"]
).astype(int)
hinos_restantes["BERT_topic"] = (
    hinos_restantes["BERT_topic"] == hino_sample["BERT_topic"]
).astype(int)
hinos_restantes = hinos_restantes[
    [
        "nome",
        "texto_limpo",
        "categoria_id",
        "word_cluster",
        "NMF_topic",
        "sent_cluster",
        "BERT_topic",
    ]
]

---

**A seguir:** Definição dos pesos e critérios para o método TOPSIS, indicando a importância relativa de cada atributo na seleção dos hinos similares.

In [3]:
similarity_matrix_words_sample = similarity_matrix_words.loc[
    hino_sample.name, hinos_restantes.index
]
similarity_matrix_sent_sample = similarity_matrix_sent.loc[
    hino_sample.name, hinos_restantes.index
]
hinos_restantes["sim_word"] = similarity_matrix_words_sample.values
hinos_restantes["sim_sent"] = similarity_matrix_sent_sample.values
hinos_restantes.head()

,nome,texto_limpo,categoria_id,word_cluster,NMF_topic,sent_cluster,BERT_topic,sim_word,sim_sent
numero,,,,,,,,,
0,AQUILO QUE FUI NÃO SOU MAIS,"AQUILO QUE FUI NÃO SOU MAIS, NEM SOU TUDO O QU...",0,0,0,0,0,0.419714,0.787951
1,O SANGUE DE JESUS TEM PODER,"O SANGUE DE JESUS TEM PODER, PODER QUE A MIM P...",0,1,0,1,1,0.570045,0.921923
2,O SANGUE DE JESUS TEM PODER PARA SALVAR,"O SANGUE DE JESUS TEM PODER O SANGUE DE JESUS,...",0,1,0,0,0,0.438065,0.603920
3,CLAMO A TI,"CLAMO A TI, Ó MEU SENHOR, CLAMO A TI, MEU SALV...",0,1,0,0,0,0.692749,0.755317
4,QUANDO TE PROSTRARES,"QUANDO TE PROSTRARES DIANTE DE DEUS, LEMBRA QU...",0,0,0,0,0,0.652970,0.674165


---

**A seguir:** Aplicação do método TOPSIS para ranquear os hinos de acordo com os critérios definidos, gerando scores de similaridade.

In [4]:
categories = [
    "categoria_id",
    "word_cluster",
    "NMF_topic",
    "sent_cluster",
    "BERT_topic",
    "sim_word",
    "sim_sent",
]
weights = [1, 2, 4, 3, 5, 6, 7] # pesos arbitrários, podem ser alterados à gosto
weights = [w / sum(weights) for w in weights] 
profit_cost = [1, 1, 1, 1, 1, 1, 1]

---

**A seguir:** Exibição dos hinos mais similares ao hino de referência, ordenados pelo score calculado pelo TOPSIS.

In [5]:
from topsis_hamedbaziyad import TOPSIS

output = TOPSIS(
    hinos_restantes[categories],
    weights,
    profit_cost,
)
scores = pd.DataFrame(output)
scores.columns = ["topsis_score"]
hinos_restantes = pd.concat([hinos_restantes, scores], axis=1).sort_values(
    by="topsis_score", ascending=False
)

---

**Fim do notebook:** Finalização do processo de seleção, com os hinos mais similares prontos para análise, exportação ou uso em aplicações.

In [6]:
print("Sugestões para o hino:", hino_sample.name, hino_sample["nome"])
hinos_restantes[["nome", "texto_limpo", "topsis_score"]].head(10)

Sugestões para o hino: 244 EU DESCI PARA O VALE DE BÊNÇÃO E PAZ


,nome,texto_limpo,topsis_score
numero,,,
230,FONTE DE ÁGUAS VIVAS,"FONTE DE ÁGUAS VIVAS, CRISTO JESUS NOS CONCEDE...",0.985037
250,SE TU PROCURAS PELA PAZ,"SE TU PROCURAS PELA PAZ, SE VAZIO ESTÁ TEU SER...",0.972445
202,HÁ UMA FONTE ABERTA NO CALVÁRIO,HÁ UMA FONTE ABERTA NO CALVÁRIO CONTRA A IMPUR...,0.963993
149,JESUS FEZ A PROMESSA,"JESUS FEZ A PROMESSA, CONOSCO ESTARIA ATÉ O FI...",0.835042
229,AO POÇO DE JACÓ CHEGOU JESUS,"AO POÇO DE JACÓ CHEGOU JESUS, ESTAVA CANSADO, ...",0.783271
274,SE EM TUA VIDA NÃO ENCONTRAS PAZ E LUZ,"SE EM TUA VIDA NÃO ENCONTRAS PAZ E LUZ, SE O C...",0.770309
288,BASTA DIZERES O QUE QUERES,BASTA DIZERES O QUE QUERES JESUS PRONTO ESTÁ A...,0.768868
207,DEUS ENVIOU SEU FILHO AMADO,DEUS ENVIOU SEU FILHO AMADO PARA MORRER EM MEU...,0.767686
261,CRISTO CAMINHAVA LENTAMENTE,"CRISTO CAMINHAVA LENTAMENTE, PASSO A PASSO COM...",0.767533
